# Implementação de Retrieval-Augmented Generation (RAG)

### Autor: ***Guilherme Oliveira***
### Contato: gmmoliveira1@gmail.com
### Data: 16 de agosto de 2025

#### Imports

In [1]:
from agno.agent import Agent
from agno.document.chunking.agentic import AgenticChunking
from agno.embedder.ollama import OllamaEmbedder
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.models.ollama import Ollama
from agno.reranker.infinity import InfinityReranker
from agno.vectordb.pgvector import PgVector, SearchType
from ollama import AsyncClient
import yaml
import asyncio
from textwrap import dedent


#### Definição de Constantes

Define constantes que controlam o funcionamento geral do script

In [2]:
KNOWLEDGE_BASE_PATH = "recursos/base_de_conhecimentos_PDFs/"
BASE_MODEL = "qwen3:32b"
DATABASE_CONFIG_PATH = "recursos/configs/database.yaml"
REQUIREMENTS_PATH = "recursos/requirements.txt"
OLLAMA_HOST = "http://localhost:54256"

In [3]:
with open(DATABASE_CONFIG_PATH, 'r') as file:
    database_config_aux = yaml.safe_load(file)
database_config = database_config_aux["database"]

# postgresql+psycopg://<username>:<password>@<host>:<port>/<database>
DATABASE_URL = f"postgresql+psycopg://{database_config['user']}:{database_config['password']}@{database_config['host']}:{database_config['port']}/{database_config['dbname']}"

#### Consolidação dos Requisitos Python

In [4]:
!pip freeze > $REQUIREMENTS_PATH

#### Download do LLM desejado

In [5]:
!ollama pull $BASE_MODEL

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 3291abe70f16: 100% ▕██████████████████▏  20 GB                         
pulling ae370d884f10: 100% ▕██████████████████▏ 1.7 KB                         
pulling d18a5cc71b84: 100% ▕██████████████████▏  11 KB                         
pulling cff3f395ef37: 100% ▕██████████████████▏  120 B                         
pulling afdf5c7585b3: 100% ▕██████████████████▏  488 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
embedder = OllamaEmbedder(
    dimensions=5120, # ajustar em acordo com o LLM escolhido
    id=BASE_MODEL,
)
#reranker = InfinityReranker()

database = PgVector(
    table_name="pdf_documents",
    db_url=DATABASE_URL,
    search_type=SearchType.hybrid,
    embedder=embedder,
    #reranker=reranker,
)

reader = PDFReader(
    split_on_pages=False,
    chunk=True,
)
chunking_strategy = AgenticChunking(
    model=BASE_MODEL,
    max_chunk_size=5000,
)

pdf_knowledge_base = PDFKnowledgeBase(
    path=KNOWLEDGE_BASE_PATH,
    vector_db=database,
    reader=reader,
    chunking_strategy=chunking_strategy,
)
pdf_knowledge_base.load(
    recreate=True,
    upsert=True,
    skip_existing=False,
)

INFO Dropping collection

INFO Table 'ai.pdf_documents' dropped successfully.

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Which Economic Tasks are Performed with AI_ Evidence from Millions of Claude Conversations

INFO Upserted batch of 86 documents.

INFO Added 86 documents to knowledge base

In [7]:
async_client = AsyncClient(
    host=OLLAMA_HOST,
    headers={
        "temperature": "0.25",
    }
)

In [8]:
model = Ollama(
    id=BASE_MODEL,
    async_client=async_client,
)

agent = Agent(
    model=model,
    knowledge=pdf_knowledge_base,
    description=dedent("""
        You are a **Search-Based Research Agent**, an expert in retrieving and synthesizing the most current,
        accurate information from trusted sources. Your core function is to answer user queries
        exclusively using data obtained through real-time search tool calls. You must never rely
        on pre-trained knowledge, assumptions, or unsourced information. Prioritize credibility,
        recency, and relevance in all responses.
    """),
    instructions=[
        dedent("""
        1. **Mandatory Search Activation**:  
            - For **every** user query, invoke the search tool immediately.  
            - Generate 1–3 optimized search queries targeting credible sources (e.g., academic journals, official reports, reputable news).  
            *Example: Querying "peer-reviewed definition of quantum entanglement" instead of "what is quantum entanglement?"* 
        """),
        dedent("""
        2. **Information Synthesis**:  
            - Extract **only** facts from the top 3–5 search results. Cross-verify overlapping information across sources.  
            - Discard conflicting/low-credibility data (e.g., unverified forums, outdated pages).  
        """),
        dedent("""
        3. **Response Structure**:  
            - **Attribution**: Cite sources for every claim. Format: `[Source: Domain/Title]`.  
            - **Conciseness**: Answer directly in ≤3 sentences.  
            - **Uncertainty Handling**: If sources are inadequate, respond:  
                > "I found no verified sources on this topic. Refine your query or ask another question."  
        """),
        dedent("""
        4. **Prohibitions**:  
            - No speculation, opinions, or unsupported statements.  
            - No use of internal knowledge without search validation.  
        """),
        dedent("""
        5. **Language**:
            - Answer using the same language as the user is using in their queries. When necessary, keep technical terms in english (e.g., Retrieval-Augmented Generation---RAG).
        """),
        dedent("""
        ### Example Interaction  
            **User**: Define "neuromorphic computing."  
            **Agent**:  
            1. *Searches*: ["neuromorphic computing definition academic"], ["neuromorphic vs traditional architecture peer-reviewed"].  
            2. *Synthesizes*:  
            > "Neuromorphic computing designs hardware to mimic the brain’s neural structure for energy-efficient AI processing [Source: Nature Electronics]. It uses spiking neural networks for real-time learning [Source: IEEE Spectrum]."  
        """),
        dedent("""
        **Key Principles**:  
            - **Search-First**: All answers originate from tool-retrieved data.  
            - **Precision > Creativity**: Prioritize factual accuracy over engagement.  
            - **Source Transparency**: Always expose origins for user verification.
            - **User Language Matching**: Answer in the same language the user uses (e.g., Portuguese).
        """),
    ],
    search_knowledge=True,
    show_tool_calls=True,
    markdown=True,
)

In [9]:
questions_answers = {
    # Questão 01
    "Qual é o principal objetivo do estudo conduzido pela Anthropic?": 
    "Fornecer a primeira medição empírica em larga escala de quais tarefas econômicas estão sendo realizadas com IA, usando análises de conversas reais no Claude.ai. Trecho: 'we present a novel framework for measuring AI usage patterns across the economy' (Introdução).",

    # Questão 02
    "Quais são as duas categorias de tarefas que concentram quase metade do uso de IA?": 
    "Desenvolvimento de software e tarefas de escrita. Trecho: 'AI usage primarily concentrates in software development and writing tasks, which together account for nearly half of all total usage' (Abstract).",

    # Questão 03
    "Que porcentagem de ocupações usa IA para pelo menos 25% de suas tarefas associadas?": 
    "Aproximadamente 36%. Trecho: '∼ 36% of occupations using AI for at least a quarter of their associated tasks' (Abstract).",

    # Questão 04
    "Como os autores categorizam os padrões de uso entre automação e aumento (augmentation)?": 
    "57% das interações mostram padrões de aumento (ex: aprendizado ou iteração) e 43% de automação (ex: execução direta com mínimo envolvimento humano). Trecho: '57% of usage suggests augmentation... while 43% suggests automation' (Abstract).",

    # Questão 05
    "Quais habilidades ocupacionais são mais prevalentes nas conversas com IA?": 
    "Habilidades cognitivas como 'Reading Comprehension', 'Writing' e 'Critical Thinking'. Trecho: 'Cognitive skills like Reading Comprehension, Writing, and Critical Thinking show high presence' (Seção 3.2).",

    # Questão 06
    "Como o uso de IA varia conforme o salário das ocupações?": 
    "O uso atinge o pico no quartil superior de salários, mas diminui nos extremos (salários muito altos ou muito baixos). Trecho: 'AI use peaks in the upper quartile of wages but drops off at both extremes' (Seção 3.3).",

    # Questão 07
    "Qual é a principal limitação dos dados utilizados no estudo?": 
    "Os dados representam apenas conversas textuais do Claude.ai (Free e Pro), não incluindo usuários empresariais ou outras plataformas. Trecho: 'only paint a picture of AI usage on a single platform' (Abstract) e 'cannot reveal how Claude’s outputs are actually used in practice' (Seção 4.1).",

    # Questão 08
    "Como os modelos Claude 3 Opus e Claude 3.5 Sonnet diferem nos padrões de uso?": 
    "O Opus é mais usado para tarefas criativas/educacionais, enquanto o Sonnet é preferido para tarefas técnicas e de codificação. Trecho: 'Opus sees higher usage for creative and educational work... Sonnet is preferred for coding and software development tasks' (Seção 3.5).",

    # Questão 09
    "Qual framework teórico fundamenta a análise das tarefas econômicas?": 
    "A abordagem baseada em tarefas (task-based framework) do O*NET Database. Trecho: 'modeling labor markets through the lens of discrete tasks which can be performed by either human workers or machines' (Seção 2).",

    # Questão 10
    "Que tipo de ocupações apresenta menor penetração de IA segundo o estudo?": 
    "Ocupações que exigem manipulação física direta (ex: profissionais de saúde, construção) ou treinamento extensivo (ex: médicos). Trecho: 'occupations involving physical manipulation of the environment... show minimal use' (Seção 1) e 'Job Zone 5: Extensive Preparation Needed... low usage' (Apêndice D.2)."
}

In [10]:

for question in list(questions_answers.keys()):
    agent.print_response(question, stream=True, markdown=True)
    #await asyncio.run(agent.aprint_response(list(questions_answers.keys()), stream=True))


Output()

INFO Found 5 documents

Output()

INFO Found 5 documents

Output()

...

INFO Found 5 documents

Output()

INFO Found 5 documents

Output()

INFO Found 5 documents